In [16]:
import pandas as pd
import numpy as np

In [9]:
berk_hist = pd.read_csv("./data/BekshireHathaway_stock_history.csv")
cpi = pd.read_excel("./data/cpi.xlsx",sheet_name="BLS Data Series")

In [31]:
berk_hist.sort_values("Date",ascending=False).head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,YearMonth,dollar_coeff
10518,2021-12-01,421535.0,424669.0,415000.0,415660.0,20,0,0,2021-12,1.144475
10517,2021-11-30,423823.0,426125.0,416546.0,416876.0,19,0,0,2021-11,1.147992
10516,2021-11-29,429048.0,432135.0,425000.0,425260.0,19,0,0,2021-11,1.147992
10515,2021-11-26,424233.0,429803.0,421091.0,427833.0,16,0,0,2021-11,1.147992
10514,2021-11-24,434643.0,437139.0,431497.0,433333.0,15,0,0,2021-11,1.147992


In [10]:
cpi.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,1913,9.8,9.8,9.8,9.8,9.7,9.8,9.9,9.9,10.0,10.0,10.1,10.0,NaN,NaN
1,1914,10.0,9.9,9.9,9.8,9.9,9.9,10.0,10.2,10.2,10.1,10.2,10.1,NaN,NaN
2,1915,10.1,10.0,9.9,10.0,10.1,10.1,10.1,10.1,10.1,10.2,10.3,10.3,NaN,NaN
3,1916,10.4,10.4,10.5,10.6,10.7,10.8,10.8,10.9,11.1,11.3,11.5,11.6,NaN,NaN
4,1917,11.7,12.0,12.0,12.6,12.8,13.0,12.8,13.0,13.3,13.5,13.5,13.7,NaN,NaN


In [11]:
# Reshape the cpi dataframe so that each row represents a month and year with its CPI value
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
melted = cpi.melt(id_vars="Year", value_vars=months, var_name="Month", value_name="CPI")

# Map month abbreviations to their corresponding two-digit numbers
month_map = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
    "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}
melted["Month_Num"] = melted["Month"].map(month_map)

# Create a date column in the format YYYY-MM
melted["Date"] = melted["Year"].astype(str) + "-" + melted["Month_Num"]

# Create the new dataframe with just the date and CPI columns – drop rows with missing CPI values if any
cpi_by_date = melted[["Date", "CPI"]].dropna().reset_index(drop=True)
cpi_by_date.head()

,Date,CPI
0,1913-01,9.8
1,1914-01,10.0
2,1915-01,10.1
3,1916-01,10.4
4,1917-01,11.7


In [ ]:
DOLLAR_IN_TERMS_DATE = cpi_by_date["Date"].max() # this means that we are having all our dollars in terms of the most recent cpi data we have
CPI_OF_REFERENCE = cpi_by_date[cpi_by_date["Date"] == DOLLAR_IN_TERMS_DATE]["CPI"].tolist()[0]
CPI_OF_REFERENCE

cpi_by_date["dollar_coeff"] = CPI_OF_REFERENCE / cpi_by_date["CPI"] # gets coefficients that is value at DOLLAR_IN_TERMS_DATE over the value of the dollar at the given time in the df



,Date,CPI,dollar_coeff
225,2025-02,319.082,1.000000
112,2025-01,317.671,1.004442
1345,2024-12,315.605,1.011017
1233,2024-11,315.493,1.011376
1121,2024-10,315.664,1.010828


In [32]:
# Extract the Year-Month from the daily date in berk_hist
berk_hist['YearMonth'] = berk_hist['Date'].str[:7]

# Map the corresponding dollar coefficient by matching YearMonth to the Date in cpi_by_date
berk_hist['dollar_coeff'] = berk_hist['YearMonth'].map(cpi_by_date.set_index('Date')['dollar_coeff'])

# Create a new dataframe with inflation adjusted columns
inflation_adjusted_berk_hist = pd.DataFrame({
    'Date': berk_hist['Date'],
    'Open_adjusted': berk_hist['Open'] * berk_hist['dollar_coeff'],
    'High_adjusted': berk_hist['High'] * berk_hist['dollar_coeff'],
    'Low_adjusted': berk_hist['Low'] * berk_hist['dollar_coeff'],
    'Close_adjusted': berk_hist['Close'] * berk_hist['dollar_coeff']
})

inflation_adjusted_berk_hist.sort_values("Date",ascending=False).head()

,Date,Open_adjusted,High_adjusted,Low_adjusted,Close_adjusted
10518,2021-12-01,482436.391669,486023.177230,474957.245644,475712.599336
10517,2021-11-30,486545.290795,489187.967713,478191.355117,478570.192381
10516,2021-11-29,492543.547484,496087.397895,487896.477039,488194.954884
10515,2021-11-26,487015.967397,493410.281225,483408.977442,491148.737555
10514,2021-11-24,498966.561105,501831.948415,495354.979183,497462.691964


In [33]:
inflation_adjusted_berk_hist.to_csv("./data/inflation_adjusted_berkshire_stocks.csv")